## Imports

In [ ]:
# Todo: import ConcreteModel from pyomo.environ
from pyomo.environ import ConcreteModel, value

# Todo: import FlowsheetBlock from idaes.core
from idaes.core import FlowsheetBlock

In [ ]:
from hfc32_emimtf2n_nrtl_VLE import HFCILParameterBlock
import idaes.logger as idaeslog

In [ ]:
import pyomo.contrib.parmest.parmest as parmest
import pandas as pd

In [ ]:
data = pd.read_csv('r32_emimtf2n_subset.csv')

In [ ]:
data

## Model Setup

In [ ]:
eps=0.0
comp_1='R32'
x_comp_1='x_R32'
comp_2='emimTf2N'
init_tau_1_2 = -0.3
init_tau_2_1 = 0.5

In [ ]:
def NRTL_model(data):
    
    #Todo: Create a ConcreteModel object
    m = ConcreteModel()
    
    #Todo: Create FlowsheetBlock object
    m.fs = FlowsheetBlock(default={"dynamic": False})
    

    #Todo: Create a properties parameter object with the following options:
    # "valid_phase": ('Liq', 'Vap')
    # "activity_coeff_model": 'NRTL'
    m.fs.properties = HFCILParameterBlock(default={"valid_phase":
                                                 ('Liq', 'Vap'),
                                                 "activity_coeff_model":
                                                 'NRTL'})
    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    
    # Fix the state variables on the state block
    # hint: state variables exist on the state block i.e. on m.fs.state_block
    
    x = float(data[x_comp_1])+eps
    
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp[comp_2].fix(1-x)
    m.fs.state_block.mole_frac_comp[comp_1].fix(x)

    # Fix NRTL specific parameters. 
    
    # non-randomness parameter - alpha_ij (set at 0.3, 0 if i=j)
    m.fs.properties.\
        alpha[comp_2, comp_2].fix(0)
    m.fs.properties.\
        alpha[comp_2, comp_1].fix(0.3)
    m.fs.properties.\
        alpha[comp_1, comp_1].fix(0)
    m.fs.properties.\
        alpha[comp_1, comp_2].fix(0.3)

    # binary interaction parameter - tau_ij (0 if i=j, else to be estimated later but fixing to initialize)
    m.fs.properties.\
        tau[comp_2, comp_2].fix(0)
    m.fs.properties.\
        tau[comp_2, comp_1].fix(init_tau_2_1)
    m.fs.properties.\
        tau[comp_1, comp_1].fix(0)
    m.fs.properties.\
        tau[comp_1, comp_2].fix(init_tau_1_2)

    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.INFO_LOW)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))

    # Set bounds on variables to be estimated
    m.fs.properties.\
        tau["benzene", "toluene"].setlb(-5)
    m.fs.properties.\
        tau["benzene", "toluene"].setub(5)

    m.fs.properties.\
        tau["toluene", "benzene"].setlb(-5)
    m.fs.properties.\
        tau["toluene", "benzene"].setub(5)

    # Return initialized flash model
    return m

In [ ]:
from idaes.core.util.model_statistics import degrees_of_freedom

m=NRTL_model(data)

degrees_of_freedom(m)